In [1]:
# Set covering problem: you need to select a certain number of blocks in order to cover all the initial sets
# Example: find the minimum number of items that together guarantee to cover something
# Definitions

# STATE = contains all the relevant informations to represent the current situation

Tile = set of points ==> it is PROBLEM_SIZE

The problem is to maximize the number of not_taken or otherwise minimize the number of taken.
In other words, covering all points at least once with the least number of tiles possible

In [2]:
from random import random
import numpy as np
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

In [3]:
PROBLEM_SIZE = 5
NUM_SETS = 12

In [4]:
from typing import Sequence
sets = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [5]:
def goal_check(state):
  #np.all = checks whether all elements are TRUE
  #reduce = applies the logical or to all elements of the specified iterable
  return np.all(reduce(np.logical_or, [sets[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [6]:
from collections import namedtuple
State = namedtuple('State', ['taken', 'not_taken'])

In [7]:
def covered(state):
    return reduce(
        np.logical_or,
        [sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

In [8]:

#TEST: max taken - max not taken 
#l'euristica che ho definito rispetta dell'ammissibilità --> ovvero è sempre minore della somma dei singoli costa da start to end 
#sommo il numero di punti coperti e lo metto negativo, in questo modo rispetta il criterio 
def h(state):  #prendo il set che ha il max di true
    return -sum(covered(state))


In [9]:
def h_3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in sets), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

In [10]:
def goal_check(state):
    # This is a binary output: did we reach a solution? True of False
    return np.all(reduce(
        np.logical_or,
        [sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def distance(state): #H 
    # This is a numeric output that counts the amount of TRUEs in the given state. 
    # TRUE means "I am covering that individual point"
    # Since the PriorityQueue is ordered in reverse, we do PROBLEM_SIZE - [...]
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [sets[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    
def print_sets():
  print("PRINTING SETS:")
  for s in range(0,NUM_SETS):
    print(f"{s}) {sets[s]}")
  print("-"*25)  
      
def g(state):
    #print("\nELEMENTI DENTRO TAKEN\n")
    #print_sets()
    return len(state.taken)
    
  

print_sets()

frontier = PriorityQueue()
#frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
oldCost=0
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        
        print("FUNC --> ", h(new_state))
        f = g(new_state) + h(new_state)
        #to see the difference between the performance of my heuristic h and the one presented ad solution we can launch with h_3
        #f = g(new_state) + h_3(new_state)

        print("\n\nCOSTO -->: " ,  f , " STATE: " ,  new_state)
        
        frontier.put((f, new_state))
    _, current_state = frontier.get()



print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles. Sets used to cover {current_state.taken})"
)
#print_sets()

PRINTING SETS:
0) [False False  True False False]
1) [False  True False  True False]
2) [False False False False False]
3) [ True  True False False False]
4) [False False False  True False]
5) [False False  True False False]
6) [ True False False False False]
7) [False False False False False]
8) [False False False False False]
9) [False False False False False]
10) [False False False False False]
11) [False False False False  True]
-------------------------
FUNC -->  -1


COSTO -->:  0  STATE:  State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11})
FUNC -->  -2


COSTO -->:  -1  STATE:  State(taken={1}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11})
FUNC -->  0


COSTO -->:  1  STATE:  State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11})
FUNC -->  -2


COSTO -->:  -1  STATE:  State(taken={3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11})
FUNC -->  -1


COSTO -->:  0  STATE:  State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11})
FUNC -->  -1


COSTO -->: 